In [1]:
import requests
from bs4 import BeautifulSoup 
import logging
import threading 
import time

In [2]:
# here we enter the name of the product we want to search 
search_product = input("please Enter the Product name")

In [3]:
#Get all pages links for all pages which contain that product
product_pages = []
for i in range(1,50):
    try:
        product_page_url = "https://www.flipkart.com/search?q=" + search_product + "&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page="+str(i)
        product_pages.append(product_page_url)
    except Exception as e :
        print(e)

In [4]:
text_pages_lst = []
def product_pages_text_source(page_link):
    text_pages_lst.append(requests.get(page_link).text)
    return text_pages_lst

thred1 = [threading.Thread(target=product_pages_text_source, args=(i,))  for i in product_pages]
for t in thred1:
    t.start()
for t in thred1:
    t.join()
    

In [5]:
html_pages_lst = []
def product_pages_html_source(text_source_link):
    html_pages_lst.append(BeautifulSoup(text_source_link, "html.parser"))
    return html_pages_lst

thred2 = [threading.Thread(target=product_pages_html_source, args=(i,))  for i in text_pages_lst]
for t in thred2:
    t.start()
for t in thred2:
    t.join()
    

In [6]:
# get all info boxes for all products in all pages
product_info_box_lst = []

def get_product_info_box(html_page):
    info_boxs = html_page.find_all("div", {"class":"_1AtVbE col-12-12"})
    del info_boxs[:2]
    del info_boxs[len(info_boxs)-3:]
    for box in info_boxs:
        product_info_box_lst.append(box)
    return product_info_box_lst


thred3 = [threading.Thread(target=get_product_info_box, args=(i,))  for i in html_pages_lst]
for t in thred3:
    t.start()
for t in thred3:
    t.join()

In [7]:
len(product_info_box_lst)

1025

In [8]:
# Get the links for all products in all product_info_box in all pages that contain that product
products_links_lst = []

def get_product_link(info_box):
    
    try:
        product_link = "https://www.flipkart.com"+info_box.div.div.div.a['href']
        products_links_lst.append(product_link)
    except Exception as e :
        pass
    return products_links_lst


thred4 = [threading.Thread(target=get_product_link, args=(i,))  for i in product_info_box_lst]
for t in thred4:
    t.start()
for t in thred4:
    t.join()

In [20]:
len(products_links_lst)

984

In [9]:
#get all review boxes in all pages for each product link
products_review_lst = []
def get_review_boxes(product_link):
    review_text = requests.get(product_link).text
    review_html = BeautifulSoup(review_text, "html.parser")
    review_boxs = review_html.find_all("div", {"class": "_16PBlm"})

    for box in review_boxs:
        products_review_lst.append(box)
    return products_review_lst

thred5 = [threading.Thread(target=get_review_boxes, args=(i,))  for i in products_links_lst]
for t in thred5:
    t.start()
for t in thred5:
    t.join()


In [10]:
len(products_review_lst)

6949

In [11]:
# get ratings and the reviews owners form all review boxes
product_ratings = []
product_reviews_writers = []

def get_ratings_writers(product_review):
    try:

        rating = product_review.div.div.div.find_all("div", {"class": "_3LWZlK _1BLPMq"})[0].text
        writer = product_review.div.div.find_all("p", {"class": "_2sc7ZR _2V5EHH"})[0].text
        product_ratings.append(rating)
        product_reviews_writers.append(writer)
    except Exception as e:
        pass

    return product_ratings, product_reviews_writers

thred6 = [threading.Thread(target=get_ratings_writers, args=(i,))  for i in products_review_lst]
for t in thred6:
    t.start()
for t in thred6:
    t.join()


In [18]:
len(product_ratings)

6036

In [19]:
scrapped_data = {}
for item in range(len(product_ratings)):
    scrapped_data[product_reviews_writers[item]] = product_ratings[item]


In [21]:
len(scrapped_data.keys())

1735